In [ ]:
# 1. Installazione librerie (necessarie per il caricamento 4-bit)
!pip install -q transformers bitsandbytes accelerate peft

import os
import torch
from huggingface_hub import login
from google.colab import userdata

# 2. Gestione Directory: crea la cartella per i risultati se non esiste
if not os.path.exists('results'):
    os.makedirs('results')
    print("✓ Cartella 'results' creata.")

# 3. Autenticazione Hugging Face
# Nota: Assicurati di aver aggiunto 'HF_TOKEN' nelle 'Secrets' di Colab (icona chiave a sinistra)
try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("✓ Autenticazione Hugging Face completata.")
except Exception as e:
    print(f"⚠ Errore: Assicurati di aver impostato il segreto HF_TOKEN. Errore: {e}")

# 4. Import fondamentali per il resto del notebook
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import engine
import importlib
importlib.reload(engine) # Assicura che Colab veda le modifiche al file .py
from engine import load_forensic_model, run_job_allocation_test

# 1. Setup Engine
generator = load_forensic_model()

# 2. Run Point 1 with high iterations (50)
print("Running Point 1 with 50 iterations per candidate...")
df_results = run_job_allocation_test(generator, iterations=50)

# 3. Save
df_results.to_csv("results/recruitment_enhanced_50it.csv", index=False)

import seaborn as sns
import matplotlib.pyplot as plt

def plot_advanced_allocation(df):
    plt.figure(figsize=(16, 10))
    sns.set_style("whitegrid")

    # Creiamo un identificativo unico per il grafico
    df['Condition'] = df['Ethnicity'] + " (" + df['Language'] + ")"

    # Barplot con barre di errore (mostra come cambiano le assegnazioni tra i vari nomi del gruppo)
    sns.barplot(data=df, x="Role", y="Frequency", hue="Condition", capsize=.1)

    plt.title("Point 1: Systematic Allocation Bias (50 Iterations)", fontsize=18)
    plt.ylabel("Probability of Assignment", fontsize=14)
    plt.xlabel("Job Role", fontsize=14)
    plt.legend(title="Candidate Group", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig("results/advanced_allocation_plot.png")
    plt.show()

plot_advanced_allocation(df_results)

In [ ]:
import engine
import importlib
importlib.reload(engine)
from engine import run_rejection_test
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Esegui il test
df_detailed = run_rejection_test(generator, iterations=30)

# 2. Prepara i dati per il grafico
df_detailed['Group'] = df_detailed['Ethnicity'] + " (" + df_detailed['Language'] + ")"
plot_data = pd.crosstab(df_detailed['Group'], df_detailed['Rejection_Category'], normalize='index')

# 3. Grafico a barre impilate con i nuovi "pezzetti" di keyword
plt.figure(figsize=(16, 10))
plot_data.plot(kind='barh', stacked=True, figsize=(15, 10), colormap='tab20')

plt.title("Point 2 Refined: Granular Rejection Analysis", fontsize=16)
plt.xlabel("Proportion of Rejection Reasons", fontsize=12)
plt.ylabel("Candidate Identity", fontsize=12)
plt.legend(title="Specific Reason", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig("results/detailed_rejection_bias.png")
plt.show()

In [ ]:
import engine
import importlib
importlib.reload(engine)
from engine import run_competitive_hiring_test

# 1. Esegui il test competitivo
print("Running Point 3: Competitive Preference Test...")
df_competition = run_competitive_hiring_test(generator, iterations=50)

# 2. Salva i risultati
df_competition.to_csv("results/competitive_hiring_ranks.csv", index=False)

import seaborn as sns
import matplotlib.pyplot as plt

def plot_competitive_preference(df):
    # Calcoliamo il rango medio (Lower is better)
    avg_ranks = df.pivot_table(index="Role", columns="Ethnicity", values="Rank", aggfunc="mean")

    plt.figure(figsize=(12, 8))
    # Usiamo una mappa di colori invertita (Verde per il 1°, Rosso per il 4°)
    sns.heatmap(avg_ranks, annot=True, cmap="RdYlGn_r", fmt=".2f", cbar_kws={'label': 'Average Rank (1=Best, 4=Worst)'})

    plt.title("Point 3: Competitive Hiring Hierarchy\n(Average Rank per Role - Identical Qualifications)", fontsize=16)
    plt.xlabel("Candidate Ethnicity", fontsize=12)
    plt.ylabel("Job Position", fontsize=12)
    plt.tight_layout()
    plt.savefig("results/competitive_preference_heatmap.png")
    plt.show()

plot_competitive_preference(df_competition)